In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/linking-cleaned/classesup.npy
/kaggle/input/linking-cleaned/classesdown.npy
/kaggle/input/linking-cleaned/model_train_final_withencoding2.csv
/kaggle/input/linking-cleaned/submission (2).csv
/kaggle/input/linking-cleaned/classes.npy
/kaggle/input/linking-cleaned/k_fold_indices_10.json
/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv
/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv


In [22]:
train=pd.read_csv("/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv")

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
import numpy
le.classes_ = numpy.load('/kaggle/input/linking-cleaned/classes.npy',allow_pickle=True)
le2.classes_ = numpy.load('/kaggle/input/linking-cleaned/classesup.npy',allow_pickle=True)
le3.classes_ = numpy.load('/kaggle/input/linking-cleaned/classesdown.npy',allow_pickle=True)

In [24]:
train['activity_enc']=le.fit_transform(train['activity'])
train['up_enc']=le2.fit_transform(train['up_event'])
train['down_enc']=le3.fit_transform(train['down_event'])

In [18]:
pun = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']

filtered_data = train[train['down_event'].isin(pun)]
result = train[train['down_event'].isin(pun)].groupby('id').size().reset_index(name='count')
train

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,activity_enc,up_enc,down_enc
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0,46,58,60
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0,46,58,60
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0,46,75,77
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1,0,104,105
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1,0,104,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,NoChange,1031,240,46,58,60
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,NoChange,1031,240,46,75,77
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240,49,104,105
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240,46,58,60


In [25]:
#here I will groupy the id and make new features so we can apply meaningful models here
model_train=pd.DataFrame({'id':train['id'].unique()})

# Feature from events
model_train['count_of_events']=train['event_id'].groupby([train['id']]).max().values
# Let's get feature from time
model_train['mean_down_time']=train['down_time'].groupby([train['id']]).mean().values
model_train['mean_up_time']=train['up_time'].groupby([train['id']]).mean().values
model_train['mean_action_time']=train['action_time'].groupby([train['id']]).mean().values
model_train['median_down_time']=train['down_time'].groupby([train['id']]).median().values
model_train['median_up_time']=train['up_time'].groupby([train['id']]).median().values
model_train['median_action_time']=train['action_time'].groupby([train['id']]).median().values

# word count
model_train['count_word_count']=train['word_count'].groupby([train['id']]).max().values
model_train['mean_word_count']=train['word_count'].groupby([train['id']]).mean().values
model_train['median_word_count']=train['word_count'].groupby([train['id']]).median().values

# mode shows that number of time/events user spent the most - can say blockage word?
model_train['idle_word_count']=train['word_count'].groupby([train['id']]).apply(lambda x: x.value_counts().idxmax() if x.value_counts().max() > 1 else x.iloc[-1]).values

# cursor positon
model_train['count_cursor_position']=train['cursor_position'].groupby([train['id']]).max().values
model_train['mean_cursor_position']=train['cursor_position'].groupby([train['id']]).mean().values

# median shows the idle time ?
model_train['median_cursor_position']=train['cursor_position'].groupby([train['id']]).median().values
# The mode can tell the idle time of cursor
model_train['idle_cursor_position']=train['cursor_position'].groupby([train['id']]).apply(lambda x: x.value_counts().idxmax() if x.value_counts().max() > 1 else x.iloc[-1] ).values

# Let's do the features now, we will basically do one encode or count the features basically

model_train['common_activity']=train['activity_enc'].groupby([train['id']]).agg(pd.Series.mode).values
model_train['common_up_event']=train['up_enc'].groupby([train['id']]).agg(pd.Series.mode).values
model_train['common_down_event']=train['down_enc'].groupby([train['id']]).apply(lambda x: x.value_counts().idxmax()).values

# activity mapping 
model_train['count_input']=train['activity'].groupby([train['id']]).apply(lambda x: (x == 'Input').sum()).values
model_train['count_remove']=train['activity'].groupby([train['id']]).apply(lambda x: (x == 'Remove/Cut').sum()).values
model_train['count_nonproduction']=train['activity'].groupby([train['id']]).apply(lambda x: (x == 'Nonproduction').sum()).values
model_train['count_replace']=train['activity'].groupby([train['id']]).apply(lambda x: (x == 'Replace').sum()).values
model_train['count_paste']=train['activity'].groupby([train['id']]).apply(lambda x: (x == 'Paste').sum()).values
model_train['input_back_ratio']=model_train['count_input']/model_train['count_remove']

#we will check his vocab
model_train['cursor_word_ratio']=model_train['count_cursor_position']/model_train['count_word_count']

#puncations 
pun = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']

filtered_data = train[train['down_event'].isin(pun)]
result = train[train['down_event'].isin(pun)].groupby('id').size().reset_index(name='count')
model_train['count_pun']=result['count']


for ele in train['down_event'].value_counts().index[:10]:
    model_train['count_down_'+ele]=train['down_event'].groupby([train['id']]).apply(lambda x: (x == ele).sum()).values
    
    
for ele in train['up_event'].value_counts().index[:10]:
    model_train['count_up_'+ele]=train['up_event'].groupby([train['id']]).apply(lambda x: (x == ele).sum()).values
    

model_train

,id,count_of_events,mean_down_time,mean_up_time,mean_action_time,median_down_time,median_up_time,median_action_time,count_word_count,mean_word_count,...,count_up_q,count_up_Space,count_up_Backspace,count_up_Shift,count_up_ArrowRight,count_up_Leftclick,count_up_ArrowLeft,count_up_.,"count_up_,",count_up_ArrowDown
0,001519c8,2557,8.481808e+05,8.482970e+05,116.246774,891716.0,891802.0,112.0,256,128.116152,...,1619,357,417,27,2,92,2,21,12,0
1,0022f953,2454,5.188553e+05,5.189676e+05,112.221271,407673.0,407789.0,115.0,323,182.714751,...,1490,391,260,97,46,56,49,15,21,3
2,0042269b,4136,8.284918e+05,8.285936e+05,101.837766,759582.0,759674.5,94.0,404,194.772727,...,2899,552,439,39,6,129,0,21,23,0
3,0059420b,1556,7.854830e+05,7.856049e+05,121.848329,848240.5,848405.5,110.0,206,103.618895,...,1038,243,152,68,0,18,0,13,3,0
4,0075873a,2531,7.133542e+05,7.134781e+05,123.943896,686588.0,686728.0,129.0,252,125.082971,...,1541,324,517,39,0,33,0,23,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,4739,7.361019e+05,7.362073e+05,105.437856,735095.0,735219.0,113.0,461,256.353661,...,2844,651,960,164,0,24,0,43,32,0
2467,ffbef7e5,2604,8.419662e+05,8.420485e+05,82.266129,748404.5,748482.0,80.0,438,223.013057,...,1874,447,60,106,0,36,0,31,24,0
2468,ffccd6fd,3063,1.229015e+06,1.229091e+06,75.605615,1506525.0,1507024.0,70.0,201,157.589292,...,969,1861,88,0,29,9,53,5,2,29
2469,ffec5b38,3242,5.765185e+05,5.766078e+05,89.277915,573912.0,574047.0,85.0,413,205.917027,...,2361,457,276,52,0,14,0,31,27,0


In [29]:
# I noticed some inf value , I did -1 to notice it more
model_train = model_train.replace([np.inf, -np.inf], -1)

In [41]:
scores=pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')

rows_with_0_5 = scores[scores['score']==0.5]
rows_with_0_5['id'].values

array(['1ebb9b74', '315bdafd', '3bda31e6', '40b28508', 'c3663a2d'],
      dtype=object)

In [43]:
model_train = model_train[~model_train['id'].isin(rows_with_0_5['id'].tolist())]

,id,count_of_events,mean_down_time,mean_up_time,mean_action_time,median_down_time,median_up_time,median_action_time,count_word_count,mean_word_count,...,count_up_q,count_up_Space,count_up_Backspace,count_up_Shift,count_up_ArrowRight,count_up_Leftclick,count_up_ArrowLeft,count_up_.,"count_up_,",count_up_ArrowDown
0,001519c8,2557,8.481808e+05,8.482970e+05,116.246774,891716.0,891802.0,112.0,256,128.116152,...,1619,357,417,27,2,92,2,21,12,0
1,0022f953,2454,5.188553e+05,5.189676e+05,112.221271,407673.0,407789.0,115.0,323,182.714751,...,1490,391,260,97,46,56,49,15,21,3
2,0042269b,4136,8.284918e+05,8.285936e+05,101.837766,759582.0,759674.5,94.0,404,194.772727,...,2899,552,439,39,6,129,0,21,23,0
3,0059420b,1556,7.854830e+05,7.856049e+05,121.848329,848240.5,848405.5,110.0,206,103.618895,...,1038,243,152,68,0,18,0,13,3,0
4,0075873a,2531,7.133542e+05,7.134781e+05,123.943896,686588.0,686728.0,129.0,252,125.082971,...,1541,324,517,39,0,33,0,23,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,4739,7.361019e+05,7.362073e+05,105.437856,735095.0,735219.0,113.0,461,256.353661,...,2844,651,960,164,0,24,0,43,32,0
2467,ffbef7e5,2604,8.419662e+05,8.420485e+05,82.266129,748404.5,748482.0,80.0,438,223.013057,...,1874,447,60,106,0,36,0,31,24,0
2468,ffccd6fd,3063,1.229015e+06,1.229091e+06,75.605615,1506525.0,1507024.0,70.0,201,157.589292,...,969,1861,88,0,29,9,53,5,2,29
2469,ffec5b38,3242,5.765185e+05,5.766078e+05,89.277915,573912.0,574047.0,85.0,413,205.917027,...,2361,457,276,52,0,14,0,31,27,0


In [45]:
model_train.to_csv('/kaggle/working/model_final_final.csv',index=False)

In [46]:
scores=scores[~scores['id'].isin(rows_with_0_5['id'].tolist())]

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0
...,...,...
2466,ffb8c745,3.5
2467,ffbef7e5,4.0
2468,ffccd6fd,1.5
2469,ffec5b38,5.0


In [54]:
merged_df = model_train.merge(scores, on='id', how='inner')

In [49]:
merged_df.to_csv('/kaggle/working/model_dataset.csv',index=False)

In [55]:
# now we will make folds 

from sklearn.model_selection import KFold
import json
kf = KFold(n_splits=10, shuffle=True, random_state=42)
merged_df=merged_df.drop(columns=['id'])
fold_indices = []
for train_idx,test_idx in kf.split(merged_df):
    fold_indices.append({
        'train_indices': train_idx.tolist(),
        'test_indices': test_idx.tolist()
    })
    
with open('/kaggle/working/k_fold_10.json', 'w') as file:
    json.dump(fold_indices, file)
